In [2]:
import math
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from scipy.stats import chi2, f, t

matplotlib.use('TkAgg')

# Данные
data = {
    'Доллар': [28.12, 28.18, 28.13, 28.08, 28.06, 28.03, 28.02, 28.00, 27.99, 27.93, 27.95, 27.97],
    'Евро': [36.13, 35.97, 35.97, 36.00, 36.13, 36.28, 36.34, 36.47, 36.54, 36.50, 36.52, 36.54],
    'Иена': [26.97, 26.80, 26.77, 26.63, 26.53, 26.70, 26.67, 26.63, 26.60, 26.50, 26.55, 26.52],
    'Фунт': [52.63, 52.32, 52.26, 52.28, 52.43, 52.58, 52.90, 52.99, 52.81, 52.89, 52.62, 52.67]
}

df = pd.DataFrame(data)
# 1) Построение матрицы парных коэффициентов корреляций
pair_correlation_matrix = df.corr()
print("Матрица парных коэффициентов корреляций:")
print(pair_correlation_matrix)
# Визуализация матрицы межфакторных корреляций
plt.figure(figsize=(8, 6))
sns.heatmap(pair_correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Матрица парных коэффициентов корреляций')
plt.show()


Матрица парных коэффициентов корреляций:
          Доллар      Евро      Иена      Фунт
Доллар  1.000000 -0.925259  0.779088 -0.717067
Евро   -0.925259  1.000000 -0.608195  0.829605
Иена    0.779088 -0.608195  1.000000 -0.301761
Фунт   -0.717067  0.829605 -0.301761  1.000000


In [3]:
# Производим отбор факторов
df = df[['Евро', 'Иена', 'Фунт']]

#--------------------------------------------------------------------
# 1) Построение матрицы межфакторных корреляций
correlation_matrix = df.corr()
print("Матрица межфакторных корреляций:")
print(correlation_matrix)

# Визуализация матрицы межфакторных корреляций
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Матрица межфакторных корреляций')
plt.show()
#--------------------------------------------------------------------

Матрица межфакторных корреляций:
          Евро      Иена      Фунт
Евро  1.000000 -0.608195  0.829605
Иена -0.608195  1.000000 -0.301761
Фунт  0.829605 -0.301761  1.000000


In [8]:
#--------------------------------------------------------------------
# Определитель матрицы
determinant = np.linalg.det(correlation_matrix)
print(f"\nОпределитель матрицы: {determinant}")

# 2) Вычисление статистики Фаррара-Глоубера
n = len(df)  # количество наблюдений
k = df.shape[1]  # количество факторов

farrar_glober_statistic = -1*(n-1-1/6*(2*k+5)) * math.log(determinant)
print(f"\nСтатистика Фаррара-Глоубера: {farrar_glober_statistic}")

# Табличное значение для Хи-квадрат
alpha = 0.05
df_chi2 = (k - 1) * (k - 1)  # степени свободы
chi2_table_value = chi2.ppf(1 - alpha, df_chi2)
print(f"\nТабличное значение Хи-квадрат: {chi2_table_value}")
#--------------------------------------------------------------------


Определитель матрицы: 0.15530835461481096
3
12

Статистика Фаррара-Глоубера: 17.071475241848983

Табличное значение Хи-квадрат: 9.487729036781154


In [5]:
#--------------------------------------------------------------------
# 3) Вычисление обратной матрицы
inverse_matrix = np.linalg.inv(correlation_matrix)
print("\nОбратная матрица:")
print(inverse_matrix)

# Визуализация обратной матрицы
plt.figure(figsize=(8, 6))
sns.heatmap(inverse_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Обратная матрица')
plt.show()

# Степени свободы
df1 = 2
df2 = 9

# Находим критическое значение F
f_critical = f.ppf(1 - alpha, df1, df2)

print(f"Критическое значение F при уровне значимости {alpha}: {f_critical}")
# Вычисление F-критериев
f_statistics = np.diag(inverse_matrix)
print("\nF-критерии:")
print(f_statistics)

# Визуализация F-критериев
plt.figure(figsize=(8, 4))
sns.barplot(x=df.columns, y=f_statistics)
plt.title('F-критерии')
plt.ylabel('Значение F-критерия')
plt.axhline(y=f_critical, color='r', linestyle='--')  # Уровень f_critical
plt.show()
#--------------------------------------------------------------------


Обратная матрица:
[[ 5.85248792  2.30414313 -4.15995365]
 [ 2.30414313  2.00733079 -1.30579483]
 [-4.15995365 -1.30579483  4.05708109]]
Критическое значение F при уровне значимости 0.05: 4.256494729093748

F-критерии:
[5.85248792 2.00733079 4.05708109]


In [20]:
#--------------------------------------------------------------------
# 4) Вычисление частных коэффициентов корреляции

# Создание матрицы частных корреляций
partial_corr_matrix = np.zeros((k, k))

# Вычисление частных коэффициентов корреляции
for i in range(k):
    for j in range(k):
        if i != j:
            partial_corr_matrix[i, j] = -inverse_matrix[i, j] / np.sqrt(inverse_matrix[i, i] * inverse_matrix[j, j])
        else:
            partial_corr_matrix[i, j] = 1  # Корреляция с самой собой равна 1

# Преобразование в DataFrame для удобства
partial_corr_df = pd.DataFrame(partial_corr_matrix, columns=['Евро', 'Иена', 'Фунт'], index=['Евро', 'Иена', 'Фунт'])

# Вывод матрицы частных коэффициентов корреляции
print("Матрица частных коэффициентов корреляции:")
print(partial_corr_df)

# Заменяем NaN на 0
partial_corr_df.fillna(0, inplace=True)

# Визуализация матрицы частных корреляций
plt.figure(figsize=(8, 6))
sns.heatmap(partial_corr_df, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Матрица частных коэффициентов корреляций')
plt.show()

ddf = n - k - 1  # Степени свободы

# Функция для расчета t-критерия
def calculate_t_statistic(r, n, k):
    return (r * np.sqrt(n - k - 1)) / np.sqrt(1 - r**2)

# Расчет t-критериев
t_statistics = pd.DataFrame(index=partial_corr_df.index, columns=partial_corr_df.columns)

for i in range(len(partial_corr_df)):
    for j in range(len(partial_corr_df)):
        if i != j:
            r = partial_corr_df.iloc[i, j]  # Используем iloc для доступа по индексам
            t_statistics.iloc[i, j] = calculate_t_statistic(r, n, k)

print("t-критерии для коэффициентов частной корреляции:")
print(t_statistics)

# Критическое значение t
alpha = 0.05
t_critical = t.ppf(1 - alpha/2, ddf)

print(f"\nКритическое значение t при уровне значимости {alpha}: {t_critical}")
#--------------------------------------------------------------------

Матрица частных коэффициентов корреляции:
          Евро      Иена      Фунт
Евро  1.000000 -0.672248  0.853712
Иена -0.672248  1.000000  0.457571
Фунт  0.853712  0.457571  1.000000
t-критерии для коэффициентов частной корреляции:
          Евро      Иена      Фунт
Евро       NaN -2.568336  4.636934
Иена -2.568336       NaN  1.455517
Фунт  4.636934  1.455517       NaN

Критическое значение t при уровне значимости 0.05: 2.306004135204166
